Rename a bunch of files using Git and renamer.

In [16]:
var execSync = require('child_process').execSync;
try {
    require.resolve('renamer');
} catch (e) {
    execSync('npm install renamer');
}

var renamer = require('renamer');
var files = renamer.expand('C:\\Users\\brian.cullinan\\Documents\\portal\\src\\**\\*.scss');
var results = renamer.replace({files: files.filesAndDirs, find: '.scss', replace: '.less'});
renamer.dryRun(results).list.forEach(r => {
    console.log(r.before);
    // rename with git instead
    execSync('git --work-tree="C:\\Users\\brian.cullinan\\Documents\\portal" --git-dir="C:\\Users\\brian.cullinan\\Documents\\portal\\.git" mv ' + JSON.stringify(r.before) + ' ' + JSON.stringify(r.after));
});

Generate a bunch of spec files.

In [33]:
var execSync = require('child_process').execSync;
try {
    require.resolve('glob');
} catch (e) {
    execSync('npm install glob');
}
var fs = require('fs');
var glob = require('glob');
var path = require('path');

var cwd = 'C:\\Users\\brian.cullinan\\Documents\\portal\\src\\app\\subscriptions';

var spec = `
import {ComponentFixture} from '@angular/core/testing';
const assert = require('assert');
import {} from 'jasmine';
import {MockBackend} from '@angular/http/testing';
import {Router} from '@angular/router';
import {Location} from '@angular/common';
import {By} from '@angular/platform-browser';
import {Core} from '../../../testing/core';
{import}

describe('{title}', function () {
    let fixture: ComponentFixture<{component}>;
    let router: Router;
    let location: Location;
    let xhr: MockBackend;
    let comp: {component};

    beforeEach(() => {
        fixture = Core.createAppWithShared([], [], [{component}]);
        comp = fixture.componentInstance;
        router = fixture.debugElement.injector.get(Router);
        xhr = fixture.debugElement.injector.get(MockBackend);
        location = fixture.debugElement.injector.get(Location);
    });

    it('should load {component} component', (done) => {
        fixture.whenStable().then(() => {
            expect(comp).not.toBeNull();
            done();
        });
    });

});
`;

var files = glob.sync('**/*component.ts', {
    ignore: '**/*.spec.ts',
    cwd: cwd
});

var titles = files.map(f => {
    var name = f.split('.')[0].split('/').pop().replace();
    var ucFirst = name.replace(/(^|\s|-)([a-z])/g , (m,p1,p2) => p1+p2.toUpperCase());
    return ucFirst.replace(/-/ig, '');
})

titles.forEach((t, i) => {
    try {
        var specPath = path.join(cwd, files[i].replace('.ts', '.spec.ts'));
        var fileStr = files[i].split('/').pop().replace('.ts', '');
        var importStr = 'import {' + t + 'Component} from \'./' + fileStr + '\';';
        fs.writeFileSync(specPath, spec
                         .replace(/\{component\}/ig, t + 'Component')
                         .replace(/\{title\}/ig, t)
                         .replace(/\{import\}/ig, importStr));
    } catch (e) {}
});


Replace package.json latest with actual latest version numbers

In [16]:
var fs = require('fs');
var path = require('path');
var cwd = 'C:\\Users\\brian.cullinan\\Documents\\portal\\';
var execSync = require('child_process').execSync;
var packages = execSync('npm list --dev', {cwd: cwd}).toString().split('\n');
var packageJson = JSON.parse(fs.readFileSync(path.join(cwd, 'package.json')));
var deps = Object.keys(packageJson.dependencies);
deps.forEach((d, k) => {
    packages.forEach((p, i) => {
        if(p.indexOf(d + '@') > -1) {
            packageJson.dependencies[d] = '~' + p.split('@').pop().replace(' deduped', '');
        }
    });
});
console.log(packages);
var devDeps = Object.keys(packageJson.devDependencies);
devDeps.forEach((d, k) => {
    packages.forEach((p, i) => {
        if(p.indexOf(d + '@') > -1) {
            packageJson.devDependencies[d] = '~' + p.split('@').pop().replace(' deduped', '');
        }
    });
});
$$.mime({'text/html': '<pre>' + JSON.stringify(packageJson, null, 4) + '</pre>'});


Find unused/misplaced translation strings

In [30]:
var execSync = require('child_process').execSync;
try {
    require.resolve('glob');
} catch (e) {
    execSync('npm install glob');
}
var glob = require('glob');
var path = require('path');
var fs = require('fs');
var translateRegex = (/[\{"]([^><"]*?)\s*\|\s*translate\s*(:\s*[^><"\{\}]*?)?["\}]/ig);
var cwd = '/Users/briancullinan/Documents/portal/src/';
//var cwd = 'C:\\Users\\brian.cullinan\\Documents\\portal\\src\\';
var files = glob.sync('**/*component.html', {
    ignore: '**/temp-demo-pages/**',
    cwd: cwd
});
var translations = files.map((f, i) => {
    var html = fs.readFileSync(path.join(cwd, f)).toString();
    let r;
    var translateStrings = [];
    while((r = translateRegex.exec(html)) !== null) {
        translateStrings[translateStrings.length] = r[1].trim().replace(/[\'{}\s]/ig, '');
    }
    var desiredKey = f
                .replace('app/', '')
                .replace(/\//ig, '.')
                .replace('.component.html', '')
                .replace(/-/ig, '').split('.');
    desiredKey.pop();
    desiredKey = desiredKey.join('.').toUpperCase();
    var result = {};
    result[desiredKey] = translateStrings;
    return result;
});

var translationKeys = Object.assign.apply({}, translations);

function findMisplaced(obj, parentKey)
{
    for (var k in obj)
    {
        if (typeof obj[k] == "object" && obj[k] !== null) {
            var newParent = typeof parentKey !== 'undefined' ? (parentKey + '.' + k) : k;
            findMisplaced(obj[k], newParent);
        } else if(typeof obj[k] == "string") {
            if(obj[k].substr(0, parentKey.length) != parentKey) {
                console.log('Misplaced key: ' + obj[k] + ' in ' + parentKey);
            }
        }
    }
}

findMisplaced(translationKeys);

// get a long list of existing keys from en.js
var enJson = JSON.parse(fs.readFileSync(path.join(cwd, 'assets', 'i18n', 'en.json')).toString());


function flattenAllKeys(obj, parentKey)
{
    var result = [];
    for (var k in obj)
    {
        if (typeof obj[k] == "object" && obj[k] !== null) {
            var newParent = typeof parentKey !== 'undefined' ? (parentKey + '.' + k) : k;
            result = result.concat(flattenAllKeys(obj[k], newParent));
        } else if(typeof obj[k] == "string"
                  && parentKey.indexOf('TEMPDEMOPAGES') === -1) {
            result[result.length] = parentKey + '.' + k;
        }
    }
    return result;
}

var allENKeys = flattenAllKeys(enJson);

// find unused keys
function getUnused() {
    allENKeys.forEach(k => {
        var parentKey = k.split('.');
        parentKey.pop();
        parentKey = parentKey.join('.');
        if(typeof translationKeys[parentKey] === 'undefined' ||
            translationKeys[parentKey].indexOf(k) === -1) {
            console.log('Unused key: ' + k);
        }
    });
}

function getMissing() {
    for(var k in translationKeys) {
        if(typeof translationKeys[k] !== 'undefined') {
            translationKeys[k].forEach(i => {
                if(allENKeys.indexOf(i) === -1) {
                    console.log('Missing key: ' + i);
                }
            });
        }
    }
}

getMissing();
getUnused();


// find strings of text in html files
var htmlWordRegex = (/>[^><]*?(\b[^><]*\b)+[^><]*?</ig)
var attrWordRegex = (/(placeholder|title|alt)\s*=\s*["]([^"]*)["]/ig);
var needTranslations = files.map((f, i) => {
    var html = fs.readFileSync(path.join(cwd, f)).toString();
    let r;
    var needTranslations = [];
    while((r = htmlWordRegex.exec(html)) !== null) {
        if(r[1].trim() !== '' && r[1].match(/\|\s*translate/ig) === null) {
            needTranslations[needTranslations.length] = r[1].trim();
        }
    }
    while((r = attrWordRegex.exec(html)) !== null) {
        if(r[2].match(/\|\s*translate/ig) === null) {
            needTranslations[needTranslations.length] = r[2].trim();
        }
    }
    return {file: f, texts: needTranslations};
}).filter(t => t.texts.length > 0);

needTranslations.forEach(t => {
    console.log('Needs translating: ' + t.texts + ' in ' + t.file);
});

Missing key: SUBSCRIPTIONS.PRODUCTS.ACTEMARKETING.PROTITLE
Missing key: SUBSCRIPTIONS.PRODUCTS.ACTEMARKETING.TEAMTITLE
Unused key: SUBSCRIPTIONS.ORDERTRACKERLAYOUT.SUBTOTAL.UPDATE
Needs translating: Enter your payment information,month,year,You will be able to review your order before your card is charged,Card number,Expiration,Year,CVV,Cardholder name,Address,Country,State,City,Postal code in app/subscriptions/billing/billing.component.html
Needs translating: Your cart (' + items in app/subscriptions/cart/cart.component.html
Needs translating: item.ChargeDescription,item.ChargeAmount,subtotal,tax,total,total,message,Your card will be charged in app/subscriptions/confirm/confirm.component.html
Needs translating: Perpetual,Annual,Cloud,On premises in app/subscriptions/marketing-entrance/act-terms.component.html
Needs translating: Choose your Act! options,Number of user licenses,Total: {{total,Add to cart in app/subscriptions/marketing-entrance/product-select-dialog.component.html
Needs 